In [6]:
import os
from pymol import cmd

# Mapping of one-letter to three-letter amino acid codes
amino_acid_map = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS', 'E': 'GLU', 
    'Q': 'GLN', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE', 'L': 'LEU', 'K': 'LYS', 
    'M': 'MET', 'F': 'PHE', 'P': 'PRO', 'S': 'SER', 'T': 'THR', 'W': 'TRP', 
    'Y': 'TYR', 'V': 'VAL', 'U': 'SEC'  # Adding SEC for selenocysteine
}

# Mutation list based on the table provided (Human -> Mouse)
mutations_list = [
    (3, 'N', 'K'), (4, 'R', 'S'), (16, 'I', 'V'), (22, 'L', 'N'), (24, 'L', 'I'),
    (25, 'N', 'D'), (27, 'E', 'G'), (29, 'Y', 'F'), (30, 'I', 'V'), (31, 'Q', 'N'),
    (33, 'K', 'Q'), (35, 'F', 'Y'), (40, 'V', 'I'), (47, 'A', 'S'), (48, 'Y', 'F'),
    (49, 'C', 'U'),  # Ensure residue 49 is SEC
    (52, 'A', 'T'), (54, 'Q', 'T'), (60, 'A', 'T'), (67, 'N', 'P'),
    (69, 'G', 'N'), (71, 'I', 'T'), (74, 'A', 'G'), (87, 'T', 'K'), (99, 'C', 'R'),
    (102, 'S', 'G'), (104, 'F', 'Y'), (107, 'S', 'N'), (119, 'E', 'D'), (120, 'K', 'N'),
    (126, 'T', 'S'), (137, 'D', 'E'), (139, 'L', 'F'), (142, 'S', 'P'), (143, 'S', 'E'),
    (144, 'Q', 'H'), (148, 'E', 'D'), (173, 'H', 'R'), (177, 'Q', 'H'), (178, 'A', 'T'),
    (181, 'S', 'R'), (182, 'T', 'I'), (184, 'K', 'Q'), (188, 'L', 'M'), (192, 'K', 'N'),
    (194, 'F', 'T'), (195, 'N', 'S')
]

# Directory to save the modified structures
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/humansec"

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to mutate residues in a given PDB file
def mutate_residues(pdb_file, mutations):
    """
    Mutate residues in a PDB file using PyMOL and save each mutation as a separate file.
    
    Parameters:
    - pdb_file: Path to the input PDB file.
    - mutations: List of mutation tuples, e.g., [(49, "S", "U")].
    """
    # Load the PDB file
    cmd.load(pdb_file, "structure")
    
    # Loop through each mutation request
    for resi, original_residue, target_residue in mutations:
        selection = f"chain X and resi {resi}"  # Apply mutation to chain X
        
        # Validate selection
        if not cmd.count_atoms(selection):
            print(f"Residue {resi} not found in the structure. Skipping mutation.")
            continue
        
        # Check if the current residue matches the intended target
        cmd.wizard("mutagenesis")
        cmd.get_wizard().do_select(selection)
        
        try:
            # Special handling for SEC at position 49
            if resi == 49 and target_residue == 'U':  # Explicitly set residue 49 to SEC
                cmd.alter(selection, "resn='SEC'")
                cmd.alter(f"{selection} and name HB3", "name='HB2'")
                cmd.alter(f"{selection} and name HB2", "name='HB1'")
                cmd.alter(f"{selection} and name HG", "name='HG1'")
                cmd.alter(f"{selection} and name H", "name='HN'")
                print(f"Residue 49 set to SEC with modified atom names.")
            else:
                three_letter_residue = amino_acid_map.get(target_residue, "UNK")
                cmd.get_wizard().set_mode(three_letter_residue)
                cmd.get_wizard().apply()
        except Exception as e:
            print(f"Error applying mutation at residue {resi}: {e}")
            cmd.set_wizard()  # Close the wizard if an error occurs
            continue

        # Close the mutagenesis wizard
        cmd.set_wizard()
        
        # Remove hydrogens
        cmd.remove("hydro")
        
        # Output file name using residue name and number (e.g., "N3K.pdb" for N->K at residue 3)
        output_file = f"{output_dir}/{original_residue}{resi}{target_residue}.pdb"
        
        # Save the mutated structure
        try:
            cmd.save(output_file, "structure", format="pdb")
            print(f"Mutation applied at residue {resi}: {original_residue} -> {target_residue}. Saved to {output_file}.")
        except Exception as save_error:
            print(f"Error saving mutation for residue {resi}: {save_error}")
        
        # Reset the structure for the next mutation
        cmd.delete("structure")
        cmd.load(pdb_file, "structure")  # Reload original structure to avoid state conflicts
    
    # Cleanup
    cmd.delete("all")

# Example usage
pdb_file_path = "/home/hp/nayanika/github/GPX6/prep_structures/original_humansec.pdb"

# Run the mutation function
mutate_residues(pdb_file_path, mutations_list)


Selected!
 ExecutiveRMSPairs: RMSD =    0.066 (4 to 4 atoms)
 Mutagenesis: 12 rotamers loaded.
 Rotamer 1/12, strain=19.24
 ExecutiveRMSPairs: RMSD =    0.066 (4 to 4 atoms)
 Mutagenesis: 17 rotamers loaded.
 Rotamer 1/17, strain=17.83
Mutation applied at residue 3: N -> K. Saved to /home/hp/nayanika/github/GPX6/prep_structures/humansec/N3K.pdb.
Selected!
 ExecutiveRMSPairs: RMSD =    0.050 (4 to 4 atoms)
 Mutagenesis: 19 rotamers loaded.
 Rotamer 5/19, strain=18.12
 ExecutiveRMSPairs: RMSD =    0.059 (4 to 4 atoms)
 Mutagenesis: 3 rotamers loaded.
 Rotamer 2/3, strain=12.32
Mutation applied at residue 4: R -> S. Saved to /home/hp/nayanika/github/GPX6/prep_structures/humansec/R4S.pdb.
Selected!
 ExecutiveRMSPairs: RMSD =    0.042 (4 to 4 atoms)
 Mutagenesis: 3 rotamers loaded.
 Rotamer 2/3, strain=25.73
 ExecutiveRMSPairs: RMSD =    0.045 (4 to 4 atoms)
 Mutagenesis: 3 rotamers loaded.
 Rotamer 2/3, strain=25.44
Mutation applied at residue 16: I -> V. Saved to /home/hp/nayanika/github/

In [ ]:
import os
from pymol import cmd

# Mapping of one-letter to three-letter amino acid codes
amino_acid_map = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS', 'E': 'GLU', 
    'Q': 'GLN', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE', 'L': 'LEU', 'K': 'LYS', 
    'M': 'MET', 'F': 'PHE', 'P': 'PRO', 'S': 'SER', 'T': 'THR', 'W': 'TRP', 
    'Y': 'TYR', 'V': 'VAL', 'U': 'SEC'  # Adding SEC for selenocysteine
}

# Mutation list (Mouse Cys -> Human Sec)
mutations_list = [
    (3, 'K', 'N'), (4, 'S', 'R'), (16, 'V', 'I'), (22, 'N', 'L'), (24, 'I', 'L'),
    (25, 'D', 'N'), (27, 'G', 'E'), (29, 'F', 'Y'), (30, 'V', 'I'), (31, 'N', 'Q'),
    (33, 'Q', 'K'), (35, 'Y', 'F'), (40, 'I', 'V'), (47, 'S', 'A'), (48, 'F', 'Y'),
    (49, 'C', 'U'),  # Ensure residue 49 is SEC
    (52, 'T', 'A'), (54, 'T', 'Q'), (60, 'T', 'A'), (67, 'P', 'N'),
    (69, 'N', 'G'), (71, 'T', 'I'), (74, 'G', 'A'), (87, 'K', 'T'), (99, 'R', 'C'),
    (102, 'G', 'S'), (104, 'Y', 'F'), (107, 'N', 'S'), (119, 'D', 'E'), (120, 'N', 'K'),
    (126, 'S', 'T'), (137, 'E', 'D'), (139, 'F', 'L'), (142, 'P', 'S'), (143, 'E', 'S'),
    (144, 'H', 'Q'), (148, 'D', 'E'), (173, 'R', 'H'), (177, 'H', 'Q'), (178, 'T', 'A'),
    (181, 'R', 'S'), (182, 'I', 'T'), (184, 'Q', 'K'), (188, 'M', 'L'), (192, 'N', 'K'),
    (194, 'T', 'F'), (195, 'S', 'N')
]

# Directory to save the modified structures
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/mousecys"

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to mutate residues in a given PDB file
def mutate_residues(pdb_file, mutations):
    """
    Mutate residues in a PDB file using PyMOL and save each mutation as a separate file.
    
    Parameters:
    - pdb_file: Path to the input PDB file.
    - mutations: List of mutation tuples, e.g., [(49, "S", "U")].
    """
    # Load the PDB file
    cmd.load(pdb_file, "structure")
    
    # Loop through each mutation request
    for resi, mouse_residue, human_residue in mutations:
        selection = f"chain X and resi {resi}"  # Apply mutation to chain X
        
        # Check if the current residue matches the intended target
        cmd.wizard("mutagenesis")
        cmd.do(f"select {selection}")
        cmd.get_wizard().do_select(selection)
        
        # Special handling for SEC at position 49
        if resi == 49 and human_residue == 'U':  # Explicitly set residue 49 to SEC
            cmd.alter(selection, "resn='SEC'")
            print(f"Residue 49 set to SEC.")
        else:
            three_letter_residue = amino_acid_map.get(human_residue, "UNK")
            cmd.get_wizard().set_mode(three_letter_residue)
            cmd.get_wizard().apply()
        
        # Close the mutagenesis wizard
        cmd.set_wizard()
        
        # Remove hydrogens
        cmd.remove("hydro")
        
        # Output file name using residue name and number (e.g., "K3N.pdb" for K->N at residue 3)
        output_file = f"{output_dir}/{mouse_residue}{resi}{human_residue}.pdb"
        
        # Save the mutated structure
        cmd.save(output_file, "structure", format="pdb")
        print(f"Mutation applied at residue {resi}: {mouse_residue} -> {human_residue}. Saved to {output_file}.")
    
    cmd.delete("all")

# Example usage
pdb_file_path = "/home/hp/nayanika/github/GPX6/prep_structures/original_mousecys.pdb"

# Run the mutation function
mutate_residues(pdb_file_path, mutations_list)


In [1]:
import os
from pymol import cmd

# Mapping of one-letter to three-letter amino acid codes
amino_acid_map = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS', 'E': 'GLU', 
    'Q': 'GLN', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE', 'L': 'LEU', 'K': 'LYS', 
    'M': 'MET', 'F': 'PHE', 'P': 'PRO', 'S': 'SER', 'T': 'THR', 'W': 'TRP', 
    'Y': 'TYR', 'V': 'VAL', 'U': 'SEC'  # Adding SEC for selenocysteine
}

# Mutation list (Mouse Cys -> Human Sec)
mutations_list = [
    (4, 'S', 'R'), (47, 'S', 'A'), (48, 'F', 'Y'), (60, 'T', 'A'), (24, 'I', 'L'),
    (52, 'T', 'A'), (54, 'T', 'Q'), (87, 'K', 'T'), (99, 'R', 'C'),
    (102, 'G', 'S'), (104, 'Y', 'F'), (107, 'N', 'S'), (142, 'P', 'S'), (143, 'E', 'S'),
    (144, 'H', 'Q'), (148, 'D', 'E'), (177, 'H', 'Q'), (178, 'T', 'A'), (139, 'F', 'L'),
    (181, 'R', 'S')
]

# Directory to save the modified structures
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/"

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to mutate residues in a given PDB file
def mutate_residues(pdb_file, mutations):
    """
    Mutate residues in a PDB file using PyMOL and save each mutation as a separate file.
    
    Parameters:
    - pdb_file: Path to the input PDB file.
    - mutations: List of mutation tuples, e.g., [(49, "C", "U")].
    """
    # Load the PDB file
    cmd.load(pdb_file, "structure")
    
    # Loop through each mutation request
    for resi, mouse_residue, human_residue in mutations:
        selection = f"chain X and resi {resi}"  # Apply mutation to chain X
        
        # Check if the current residue matches the intended target
        cmd.wizard("mutagenesis")
        cmd.do(f"select {selection}")
        cmd.get_wizard().do_select(selection)
        
        # Special handling for SEC at position 49
        if resi == 49 and human_residue == 'U':  # Explicitly set residue 49 to SEC
            cmd.alter(selection, "resn='SEC'")
            cmd.alter(f"{selection} and name HB3", "name='HB2'")
            cmd.alter(f"{selection} and name HB2", "name='HB1'")
            cmd.alter(f"{selection} and name HG", "name='HG1'")
            cmd.alter(f"{selection} and name H", "name='HN'")
            print(f"Residue 49 set to SEC with modified atom names.")
        else:
            three_letter_residue = amino_acid_map.get(human_residue, "UNK")
            cmd.get_wizard().set_mode(three_letter_residue)
            cmd.get_wizard().apply()
        
        # Close the mutagenesis wizard
        cmd.set_wizard()
        
        # Remove hydrogens
        cmd.remove("hydro")
        
        # Output file name using residue name and number (e.g., "K3N.pdb" for K->N at residue 3)
        output_file = f"{output_dir}/{mouse_residue}{resi}{human_residue}.pdb"
        
        # Save the mutated structure
        cmd.save(output_file, "structure", format="pdb")
        print(f"Mutation applied at residue {resi}: {mouse_residue} -> {human_residue}. Saved to {output_file}.")
    
    cmd.delete("all")

# Example usage
pdb_file_path = "/home/hp/nayanika/github/GPX6/"

# Run the mutation function
mutate_residues(pdb_file_path, mutations_list)


PyMOL>select chain X and resi 4
 Selector: selection "sele" defined with 11 atoms.
Selected!
 Mutagenesis: 3 rotamers loaded.
 Rotamer 2/3, strain=17.05
 ExecutiveRMSPairs: RMSD =    0.041 (4 to 4 atoms)
 Mutagenesis: 27 rotamers loaded.
 Rotamer 1/27, strain=16.48
Mutation applied at residue 4: S -> R. Saved to /home/hp/nayanika/github/GPX6/prep_structures/C49U_T60A_I24L/S4R.pdb.
PyMOL>select chain X and resi 47
Selected!
 Selector: selection "sele" defined with 6 atoms.
 ExecutiveRMSPairs: RMSD =    0.084 (4 to 4 atoms)
 Mutagenesis: 21 rotamers loaded.
 Rotamer 17/21, strain=18.03
 ExecutiveRMSPairs: RMSD =    0.088 (4 to 4 atoms)
 Mutagenesis: no rotamers found in library.
Mutation applied at residue 47: S -> A. Saved to /home/hp/nayanika/github/GPX6/prep_structures/C49U_T60A_I24L/S47A.pdb.
PyMOL>select chain X and resi 48
Selected!
 Selector: selection "sele" defined with 11 atoms.
 ExecutiveRMSPairs: RMSD =    0.041 (4 to 4 atoms)
 Mutagenesis: no rotamers found in library.
 Exec